In [0]:
import pandas as pd
import io
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from google.colab import files

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
!kill -9 -1

In [5]:
uploaded_sample = files.upload() 

In [4]:
uploaded_train = files.upload()

Saving train.csv to train.csv


In [7]:
uploaded_test = files.upload()

Saving test.csv to test.csv


In [7]:
sample_submission_df = pd.read_csv(io.StringIO(uploaded_sample['sampleSubmission.csv'].decode('utf-8')))
print('shape of sample submission df: {shape}'.format(shape= sample_submission_df.shape ))

shape of sample submission df: (66292, 2)


In [10]:
sample_submission_df.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [5]:
input_df = pd.read_csv(io.StringIO(uploaded_train['train.csv'].decode('utf-8')))
print('shape of input df: {shape}'.format(shape= input_df.shape ))

shape of input df: (156060, 4)


In [8]:
input_test_df = pd.read_csv(io.StringIO(uploaded_test['test.csv'].decode('utf-8')))
print('shape of input test df: {shape}'.format(shape= input_test_df.shape ))

shape of input test df: (66292, 3)


In [9]:
input_test_df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [10]:
input_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


**The sentiment labels are:**

0 - negative </br>
1 - somewhat negative </br>
2 - neutral </br>
3 - somewhat positive </br>
4 - positive </br>

In [11]:
X_train = input_df.drop('Sentiment', axis=1)
Y_train = input_df.Sentiment
print('shape of X train {0}, \nshape of Y train {1}, \nX train head \n{2}, \nY train head \n{3}'.format(X_train.shape, Y_train.shape, X_train.head(), Y_train.head()))

shape of X train (156060, 3), 
shape of Y train (156060,), 
X train head 
   PhraseId  SentenceId                                             Phrase
0         1           1  A series of escapades demonstrating the adage ...
1         2           1  A series of escapades demonstrating the adage ...
2         3           1                                           A series
3         4           1                                                  A
4         5           1                                             series, 
Y train head 
0    1
1    2
2    2
3    2
4    2
Name: Sentiment, dtype: int64


In [12]:
Y_train = Y_train.values.reshape(-1,1)
Y_train.shape

(156060, 1)

In [13]:
maxLen = len(max(X_train.Phrase, key=len).split())
maxLen

51

In [0]:
def convert_to_one_hot(Y,c):
  '''
  convert a numpy array into one hot vector.
  one hot vector means an array where at particular position the value is 1 and rest are 0
  
  Arguments:
  Y -- numpy array to determine the position where value is 1
  c -- output shape of the array
  
  Returns:
  array_o_h -- one hot vector of shape (c,1)
  '''
  
  array_o_h = np.eye(c)[Y.reshape(-1)]
  return array_o_h

In [34]:
'''
Example of running convert_to_one_hot
a = np.array([range(10)])
print(a, '\n')
print (convert_to_one_hot(a,10))'''

(array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]), '\n')
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [15]:
Y_oh_train = convert_to_one_hot(Y_train, c = 5) # 5 because total number of unique sentiments are 5
Y_oh_train.shape

(156060, 5)

In [16]:
print ('{0} is converted into one hot {1}'.format(Y_train[10], Y_oh_train[10]))

[2] is converted into one hot [0. 0. 1. 0. 0.]


##Word Embeddings ##

represent each word with the feature vector. for example, we have words: man, woman, apple, football and the four features are male, female, fruit and sports. By properties, the value male will be more for man compare to others. The main advantage of using this to find the relation between words and take decision based on the relations. <br/>

**Embedding Matrix**

Embedding matrix is the outcome of the learning word embedding. 

For example, word is encoded in a dictionary represented by the one hot vector of shape (m___word, 1) and the embedding vector containg the features (n_features, 1)  then the algorithm will generate the matrix (n_features, m__word) and each word has now have a vector of shape (n_features, 1)


###Learning Word embeddings###

**Natural language model**:

Sentence Example: I want to play football in the _ .

We have to find out the words in the blank. To pick up the word we need to learn the language model. 

Steps:
1. Generate the embedding vector
        a. Caculate the dot vector (e_j) between featurised vector with the word vector
2. Pass the Embedding  vector  to the NN layer
3. Final layer is the softmax laer from where we get the max value of one word which is our target word

** Alternative approaches:**
Pick up the words for context like cosider a window having 4 words before and 4 words later or last 1 word and many more. 

Many models are already in market like Word2Vec and GloVe 

###Word2Vec model:

* Vocabulary size = 10,000 words
* Let's say that the context word are c and the target word is t
* We want to learn c to t
* We get ec by E. oc
* We then use a softmax layer to get P(t|c) which is ŷ
* Also we will use the cross-entropy loss function.

This model is called skip-grams model. the problem is with the softmax layer  where adding takes a lot of times to execute. To overcome this challenge, another approach known as 'Negative Sampling'. 

### Negative Sampling

For example, the context word is football then for word game it will be 1 and the rest are 0 like for juice, king, book etc. 

The task is then:

> *  Pick a positive context
> * Pick K negative context from the dictionary.

**K** is the  hyperparameter. k is recommended to be from 5 to 20 in small datasets and for larger case -2 to 5

**Advantage of the Negative Sampling: **

Now the problem is a binary classification. 

** Glove word Vectors **

Another approach. 

** Important line **

Xct = # times t appears in context of c

### Conclusions on word embeddings

*  it requires a lot of time to train
* Better use pre loaded model











In [17]:
uploaded_glove_6B_50d = files.upload()

Saving glove.6B.50d.txt to glove.6B.50d.txt


In [0]:
def read_glove_vecs(glove_file):
  '''
  Convert the word embediing into a dictionary format where key is the word and value are the embedding vector of each corresponding file also tagged the word with the resective integer
  Arguments:
  glove_file -- pre-trained 50 dimensional Glove vectors to represent words 
  Return:
  a tuple of 
    -- word_to_index : word to integer
    -- index_to_word : integer to word
    -- word_to_vec_map : dictionary containing word with the embedding vector value
  '''
  print(glove_file)
  with open(glove_file, 'r') as f:
    words = set()
    words_to_vec_map = {}
    for line in f:
      line = line.strip().split()
      curr_word = line[0]
      words.add(curr_word) # hence its set by default it will not allowed to insert one element more than one
      words_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
      
    i = 1
    words_to_index = {}
    index_to_words = {}
    for w in sorted(words):
      words_to_index[w] = i
      index_to_words[i] = w
      i = i + 1
  
  return words_to_index, index_to_words, words_to_vec_map

In [19]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(uploaded_glove_6B_50d.keys()[0])

glove.6B.50d.txt


In [21]:
len(word_to_index), len(index_to_word), len(word_to_vec_map)

(400000, 400000, 400000)

In [22]:
input_sentences_df = X_train.Phrase
input_sentences_df.head(10)

0    A series of escapades demonstrating the adage ...
1    A series of escapades demonstrating the adage ...
2                                             A series
3                                                    A
4                                               series
5    of escapades demonstrating the adage that what...
6                                                   of
7    escapades demonstrating the adage that what is...
8                                            escapades
9    demonstrating the adage that what is good for ...
Name: Phrase, dtype: object

In [0]:
def sentence_to_avg(sentence, word_to_vec_map):
  '''
  Converts a sentence (string) into a list of words (strings).Ignore the most common used word by suing nltk corpus stop words.Extracts the GloVe representation of each word and averages its value 
  into a single vector encoding the meaning of the sentence
  
  Arguments:
  sentence -- string, one training example generally one single sentence
  word_to_vec_map -- dictionary maps every word in a vocabulary into 50 dimensional vector representation
  
  Returns:
  avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
  '''
  
  words = word_tokenize(sentence)
  stop_words = set(stopwords.words('english'))
  filtered_sentence = [w for w in words if not w in stop_words]
  avg = np.zeros((50,))
  
  for w in filtered_sentence:
    avg += word_to_vec_map[w]
  avg = avg/ len(words)
  
  return avg

In [24]:
sentence_to_avg('everything now i do is getting tough!', word_to_vec_map).shape

['everything', 'getting', 'tough', '!']


(50,)

In [0]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [0]:
def forward_propogation(W, avg, b):
  '''
  compute the forward propagation. Here the total number of layer is 1. After calculating the (weight * avg value(shape of 50,) + bias), the result is passed through the softmax layer and get the value in between 0-5 
  
  Arguments:
  W -- weight value 
  avg -- avg value of the sentence
  b -- bias value
  
  Return:
  a -- activation value of the layer
  '''
  z = np.dot(W,avg) + b
  a = softmax(z)
  
  return a

In [0]:
def compute_gradient(Y_pred, Y_oh, n_y, n_h, avg):
  '''
  Compute the gradient of the parameters.
  
  Arguments:
  Y_pred -- predicted values 
  Y_oh -- original values
  n_y -- number of the output classes
  n_h -- dimension of the glove vector
  
  Return: 
  dz -- gradient of the activation value
  dW -- gradient of the weight value
  db -- gradient of the bias direction
  '''
  dz = Y_pred - Y_oh
  dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
  db = dz

  return dz, dW, db

In [0]:
def update_parameteres(W, learning_rate, dW, b, db):
  '''
  update the parameters of the parameters
  
  Arguments:
  W -- current weight value
  learning_rate -- how fast the parameter will learn
  dW -- gradient descent value of weight
  b -- current bias value
  db -- gradient descent of the bias value
  
  Returns:
  W -- update value of Weight
  b -- update value of bias
  '''
  W = W - learning_rate * dW
  b = b - learning_rate * db
  return W, b

In [0]:
def predict (X, Y, W, b, word_to_vec_map):
  '''
  Given X (sentences) and Y (labels), predict sentiment and compute the accuracies of the model over the given set
  
  Arguments:
  X -- input data containing sentences, numpy array of shape (m, None)
  Y -- labels, containing index of the label, numpy array of shape (m, 1)
  W -- final value of weights parameter
  b -- final value of bias
  word_to_vec_map -- dictionalry mapping every word in a vocabulary into its 50-dimensional vector representation
  Returns:
  pred -- numpy array of shape (m, 1) with your predictions
  '''
  m = X.shape[0]
  pred = np.zeros((m,1))
  
  for j in range(m): # Loop over training examples   
    # Split jth test example (sentence) into list of lower case words
    words = X[j].lower().split()
    
    # Average words vector
    avg = sentence_to_avg(sentence, word_to_vec_map)
    activation = forward_propogation(W, avg, b)
    pred[j] = np.argmax(A)
    
  print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
  
  return pred
    

In [0]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations= 400):
  '''
  Model to train word vector representation in numpy.
  
  Arguments:
  X -- input data, numpy array of sentences as strings, of shape (m,1)
  Y -- labels, numpy array of integers between 0 and 5, numpy-array of shape (m,1)
  word_to_vec_map -- dictionalry mapping every word in a vocabulary into its 50-dimensional vector representation
  learning_rate -- learning rate for the stochastic gradient descent algorithm
  num_iteration -- number of iterations
  
  Returns:
  pred -- vector of predictions, numpy-array of shape (m,1)
  W -- weight matrix of the softmax layer, of shape (n_y, n_h)
  b -- bias of the softmax layer, of shape (n_y,)
  '''
  np.random.seed(1)
  
  # Define number of training example
  m = Y.shape [0]    # number of training examples
  n_y = 5            # number of classes
  n_h = 50           # dimensions of the GloVe vectors
  
  # Initialize parameters using Xavier initialization
  W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
  b = np.zeros((n_y,))
  
  # Convert Y to Y_onehot with n_y classes
  Y_oh = convert_to_one_hot(Y, c = n_y)
  
  # Optimization loop
  for t in range (num_iterations):  # Loop over the number of iterations
    for i in range(m):              # Loop over the training examples
      
      ### Start coed here ######
      # Average the word vectrs of the words from the i'th training_example
      avg = sentence_to_avg(sentence=X[i].lower(), word_to_vec_map = word_to_vec_map)
      
      # Forward propagate eh avg through the softmax layer
      '''   
      z = np.dot(W,avg) + b
      a = softmax(z)
      '''
      a = forward_propogation(W, avg, b)
      
      # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
      cost = -np.sum(np.multiply(Y_oh[i], np.log(a))) 
      ### END CODE HERE ###
      
      # Compute gradients
      dz, dW, db = compute_gradient(a, Y_oh[i], n_y, n_h,avg)
      
      # Update parameters with Stochastic Gradient Descent
      '''      
      W = W - learning_rate * dW
      b = b - learning_rate * db
      '''
      W, b = update_parameteres(W, learning_rate, dW, b, db)
    print (t)
    if t % 100 == 0:
      print("Epoch: " + str(t) + " --- cost = " + str(cost))
      pred = predict(X, Y, W, b, word_to_vec_map)
      
  return pred,  W, b

In [34]:
pred, W, b = model(input_sentences_df.values, Y_train, word_to_vec_map)
print(pred)

KeyError: ignored